# Clustering Crypto

In [27]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [28]:
# 1. Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.tail(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1242,GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
1243,SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
1251,PUNK,SteamPunk,PoS,False,PoS,NaN,40000000


In [29]:
# list data types
crypto_df.dtypes

Unnamed: 0          object
CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [30]:
# 2. Keep all the cryptocurrencies that are being traded.
# crypto_df.count()

crypto_df = crypto_df[crypto_df["IsTrading"] == True]
crypto_df.head(10)


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [31]:
# 3. Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df[crypto_df["Algorithm"] != ""]
crypto_df.head(10)


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [32]:
crypto_df.shape

(1144, 7)

In [33]:
# 4. Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(['IsTrading'], axis=1)

In [34]:
# Find null values
# Using LIST COMPREHENSION

for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")


Column Unnamed: 0 has 0 null values
Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [35]:
# 5. Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

# Verify dropped null values
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")


Column Unnamed: 0 has 0 null values
Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values


In [36]:
crypto_df.shape

(685, 6)

In [37]:
# 6. Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]

#Make unnamed Column the Index
crypto_df = crypto_df.set_index('Unnamed: 0', drop=True)

crypto_df.head(10)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [38]:
crypto_df.shape

(532, 5)

In [39]:
#7. Create a new DataFrame that holds only the cryptocurrencies names.
cryptocurrencies_names_df = crypto_df["CoinName"]
cryptocurrencies_names_df.head()

Unnamed: 0
42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [40]:
# 8. Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(['CoinName'], axis=1)
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000


In [41]:
crypto_df.shape

(532, 4)

In [42]:
# 9. Use get_dummies() to create variables for text features.

# For categorical columns, use 
# **Pandas get_dummies()** for binary coding

# Create our features
X = crypto_df.copy()
X = pd.get_dummies(X, columns=["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# 10. Standardize the data with StandardScaler().

X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:4])


[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

In [44]:
X_scaled.shape

(532, 98)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [45]:
# 2. Using PCA to reduce dimension to three principal components.

from sklearn.cluster import AgglomerativeClustering

# Initialize PCA model
pca = PCA(n_components=3)


# Get 3 principal components for the X_scaled data.
X_scaled_pca = pca.fit_transform(X_scaled)

X_scaled_pca

array([[-0.33354223,  1.03764173, -0.59228413],
       [-0.31687542,  1.03787602, -0.59278091],
       [ 2.29071598,  1.70413447, -0.77270767],
       ...,
       [ 0.32534142, -2.28532292,  0.45244907],
       [-0.18906614, -2.06466885,  0.37089096],
       [-0.27770632,  0.78416652, -0.24022776]])

In [46]:
# 3. Create a DataFrame with the three principal components.

# Transform resulting PRINCIPAL COMPONENTS to a DataFrame to fit K-Means
pcs_df = pd.DataFrame(data = X_scaled_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)

pca.explained_variance_ratio_

array([0.02792975, 0.02139929, 0.02049843])

In [52]:
pcs_df.head(10)

,PC 1,PC 2,PC 3
Unnamed: 0,,,
42,-0.333542,1.037642,-0.592284
404,-0.316875,1.037876,-0.592781
1337,2.290716,1.704134,-0.772708
BTC,-0.145934,-1.324525,0.171526
ETH,-0.145205,-2.054052,0.411336
LTC,-0.160898,-1.125678,0.017083
DASH,-0.392694,1.272733,-0.562363
XMR,-0.155516,-2.238042,0.466637
ETC,-0.143647,-2.054140,0.411307


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [97]:
# 2. Create an elbow curve to find the best value for K.

# a. Store Values of K to Plot
inertia = []
k = list(range(1, 11))



# b. Loop Through K  Values and Find Inertia
# Looking for the best K by looping 
# through each K value, find the inertia, 
# and store it into the inertia list.
for i in k:
    km = KMeans(n_clusters=i, random_state=0) 
    km.fit(pcs_df)                        
    inertia.append(km.inertia_)

    
    
#c. Create a DataFrame and Plot the Elbow Curve using hvPlot    
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)  


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [98]:
# 3. Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)


# Fit the model
model.fit(pcs_df)


# Predict clusters
predictions = model.predict(pcs_df)

print(predictions)

[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 0 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 3 0
 3 3 0 3 3 3 3 0 1 1 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 3 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [99]:
# 4. Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

# 5. Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df,cryptocurrencies_names_df], axis = 1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# pcs_df["class"] = model.labels_
clustered_df["class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)


(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.333542,1.037642,-0.592284,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.316875,1.037876,-0.592781,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.290716,1.704134,-0.772708,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145934,-1.324525,0.171526,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.145205,-2.054052,0.411336,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.160898,-1.125678,0.017083,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.392694,1.272733,-0.562363,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.155516,-2.238042,0.466637,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.143647,-2.054140,0.411307,Ethereum Classic,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [112]:
# 2. Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
color="class",
    symbol="class",
    width=800,
# 3. CoinName and Algorithm columns to hover_name and hover_data
#     https://plotly.com/python-api-reference/generated/plotly.express.scatter_3d.html#plotly-express-scatter-3d
hover_data=['CoinName', 'Algorithm'], 
hover_name="CoinName: "+ clustered_df['CoinName'] + " , Algorithm: "+ clustered_df['Algorithm']
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [113]:
# 4. Create a table with tradable cryptocurrencies.
# https://hvplot.holoviz.org/reference/pandas/table.html#table

clustered_df.hvplot.table(columns=
                ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply',
                 'TotalCoinsMined', 'class'], 
                          sortable=True, selectable=True)


:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [114]:
# 5. Print the total number of tradable cryptocurrencies.
print(len(clustered_df))

532


In [115]:
# 6. Scaling data to create the scatter plot with tradable cryptocurrencies.
# Use the MinMaxScaler().fit_transform method to scale 
# the TotalCoinSupply and TotalCoinsMined columns between 
# the given range of zero and one.

Scaled_df = clustered_df[["TotalCoinSupply", "TotalCoinsMined"]]
Scaled_df.head()

,TotalCoinSupply,TotalCoinsMined
Unnamed: 0,,
42,42,4.199995e+01
404,532000000,1.055185e+09
1337,314159265359,2.927942e+10
BTC,21000000,1.792718e+07
ETH,0,1.076842e+08


In [116]:
Scaled_d = MinMaxScaler().fit_transform(Scaled_df)
Scaled_data = Scaled_d
Scaled_data

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [123]:
# 7. Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
Scaled_data_df = pd.DataFrame(data=Scaled_data, index=clustered_df.index )
Scaled_data_df = Scaled_data_df.rename(columns={0: "TotalCoinSupply", 1: "TotalCoinsMined"})


# 8. Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
Scaled_data_df = pd.concat([Scaled_data_df,clustered_df["CoinName"]], axis = 1)


# 9. Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
Scaled_data_df = pd.concat([Scaled_data_df,clustered_df["class"]], axis = 1)
Scaled_data_df.head(20)


,TotalCoinSupply,TotalCoinsMined,CoinName,class
Unnamed: 0,,,,
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3


In [124]:
# 10. Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
Scaled_data_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="class",
)



:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)